In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요

#@markdown **Ngrok**
Ngrok_Key = '' #@param {type:"string"}

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/output

%cd /content

!apt -y install -qq aria2
!pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install xformers==0.0.20

!git clone -b v1.5.0 https://github.com/neuralninja22/sd-webui /content/sdd-webui
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/sdd-webui/models/ESRGAN -o 4x-UltraSharp.pth
%cd /content/sdd-webui
!git reset --hard

!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/config_default.json -O /content/sdd-webui/config.json
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/ui-config_stablesr.json -O /content/sdd-webui/ui-config.json
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/install_stablesr.sh -O /content/sdd-webui/install_stablesr.sh
!bash /content/sdd-webui/install_stablesr.sh

# 구글드라이브 연결
if Google_Drive:
  !rm /content/sdd-webui/outputs
  !mv /content/sdd-webui/outputs /content/sdd-webui/outputs_backup
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/output /content/sdd-webui/outputs
else:
  !rm /content/sdd-webui/outputs

if Ngrok_Key:
  !python launch.py --xformers --no-half-vae --theme dark --ngrok {Ngrok_Key}
else:
  !python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue
